# 💫 Segmented Spacetime Solar System Visualization Demo

This notebook demonstrates the segmented spacetime visualization framework for the Solar System.

## Theory Overview

Based on the Casu & Wrede theoretical framework:
- **Segment density**: $N(x) = \sum_i \gamma_i \cdot K_i(||x - x_i||)$
- **Time dilation**: $\tau(x) = \phi^{-\alpha \cdot N(x)}$
- **Refractive index**: $n(x) = 1 + \kappa \cdot N(x)$

Where $\phi = \frac{1+\sqrt{5}}{2}$ is the golden ratio.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Import segmented spacetime modules
from src import (
    build_icosphere, 
    SegmentedSpacetimeField,
    create_solar_system_field,
    get_complete_solar_system,
    SegmentedSpacetimeVisualizer,
    create_solar_system_orbits
)

print("✅ Modules imported successfully!")

## 1. Generate Spacetime Mesh

Create an icosphere mesh to represent the segmented spacetime structure.

In [ ]:
# Generate icosphere mesh
print("Generating spacetime mesh...")
vertices, faces = build_icosphere(radius=50.0, subdivisions=4)

print(f"Mesh: {len(vertices)} vertices, {len(faces)} faces")
print(f"Radius range: {np.min(np.linalg.norm(vertices, axis=1)):.2f} - {np.max(np.linalg.norm(vertices, axis=1)):.2f} AU")

## 2. Fetch Solar System Data

Get planetary positions and properties for field calculation.

In [ ]:
# Fetch Solar System data
print("Fetching Solar System data...")
catalog, positions = get_complete_solar_system(
    epoch="2025-01-01",
    include_asteroids=True,
    include_moons=False
)

print(f"\nSolar System catalog: {len(catalog)} bodies")
print(catalog[['name', 'mass_kg', 'radius_km', 'a_AU']].head(10))

## 3. Setup Segmented Spacetime Field

Initialize the field calculator with Solar System bodies.

In [ ]:
# Create field calculator
print("Setting up segmented spacetime field...")

# Field parameters
params = {
    'p': 2.0,           # Power-law index
    'N_bg': 0.0,        # Background density
    'N_max': 5.0,       # Maximum density
    'alpha': 1.0,       # Time dilation coupling
    'kappa': 0.015      # Refractive index coupling
}

field = SegmentedSpacetimeField(params)

# Add bodies to field
for i, row in catalog.iterrows():
    position = positions[i]
    
    # Mass scaling
    mass_solar = row['mass_kg'] / 1.98847e30
    radius_au = row['radius_km'] / 149597870.7
    
    # Natural boundary parameters
    r0 = max(radius_au, 1e-6)
    r_nb = r0 * 4
    delta = r0 * 0.8
    
    gamma = 1.0 if row['name'] == 'Sun' else min(mass_solar, 1.0)
    
    field.add_body(
        position=position,
        mass_scaled=mass_solar,
        gamma=gamma,
        r0=r0,
        r_nb=r_nb,
        delta=delta,
        name=row['name']
    )

print(f"Added {len(catalog)} bodies to field calculator")

## 4. Compute Segmented Spacetime Fields

Calculate N(x), τ(x), and n(x) on the mesh vertices.

In [ ]:
# Compute fields on mesh
print("Computing segmented spacetime fields...")
N, tau, n = field.compute_all_fields(vertices)

# Field statistics
field_stats = pd.DataFrame({
    'Field': ['N(x)', 'τ(x)', 'n(x)'],
    'Min': [N.min(), tau.min(), n.min()],
    'Max': [N.max(), tau.max(), n.max()],
    'Mean': [N.mean(), tau.mean(), n.mean()],
    'Std': [N.std(), tau.std(), n.std()]
})

print("\nField Statistics:")
print(field_stats)

## 5. Visualize Field Distributions

Plot histograms of the field values.

In [ ]:
# Create field distribution plots
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=['Segment Density N(x)', 'Time Dilation τ(x)', 'Refractive Index n(x)'],
    specs=[[{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}]]
)

# N(x) histogram
fig.add_trace(
    go.Histogram(x=N, nbinsx=50, name='N(x)', marker_color='turbo'),
    row=1, col=1
)

# τ(x) histogram  
fig.add_trace(
    go.Histogram(x=tau, nbinsx=50, name='τ(x)', marker_color='viridis'),
    row=1, col=2
)

# n(x) histogram
fig.add_trace(
    go.Histogram(x=n, nbinsx=50, name='n(x)', marker_color='plasma'),
    row=1, col=3
)

fig.update_layout(
    title="Segmented Spacetime Field Distributions",
    showlegend=False,
    height=400
)

fig.show()

## 6. Create 3D Visualization

Generate the interactive 3D segmented spacetime mesh.

In [ ]:
# Create visualizer
print("Creating 3D visualization...")

visualizer = SegmentedSpacetimeVisualizer(
    title="Segmented Spacetime — Solar System Demo"
)

# Add mesh with segment density field
field_data = {'N': N, 'tau': tau, 'n': n}
visualizer.add_mesh(vertices, faces, field_data, default_field='N')

# Add celestial bodies
visualizer.add_bodies(catalog, positions)

# Create and display figure
fig = visualizer.create_figure(width=1000, height=700)
fig.show()

print("✅ 3D visualization created!")

## 7. Field Analysis Along Radial Profiles

Analyze how fields vary with distance from the Sun.

In [ ]:
# Create radial profile points
r_points = np.logspace(-1, 2, 100)  # 0.1 to 100 AU
radial_points = np.column_stack([r_points, np.zeros_like(r_points), np.zeros_like(r_points)])

# Compute fields along radial profile
N_radial, tau_radial, n_radial = field.compute_all_fields(radial_points)

# Create radial profile plot
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Segment Density N(r)', 'Time Dilation τ(r)', 'Refractive Index n(r)', 'All Fields (log scale)'],
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# N(r)
fig.add_trace(
    go.Scatter(x=r_points, y=N_radial, mode='lines', name='N(r)', line=dict(color='red')),
    row=1, col=1
)

# τ(r)
fig.add_trace(
    go.Scatter(x=r_points, y=tau_radial, mode='lines', name='τ(r)', line=dict(color='blue')),
    row=1, col=2
)

# n(r)
fig.add_trace(
    go.Scatter(x=r_points, y=n_radial, mode='lines', name='n(r)', line=dict(color='green')),
    row=2, col=1
)

# All fields (log scale)
fig.add_trace(
    go.Scatter(x=r_points, y=N_radial, mode='lines', name='N(r)', line=dict(color='red')),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=r_points, y=tau_radial, mode='lines', name='τ(r)', line=dict(color='blue')),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=r_points, y=n_radial-1, mode='lines', name='n(r)-1', line=dict(color='green')),
    row=2, col=2
)

# Update axes
fig.update_xaxes(title_text="Distance (AU)", type="log", row=2, col=2)
fig.update_yaxes(title_text="Field Value", type="log", row=2, col=2)

for row in [1, 2]:
    for col in [1, 2]:
        if not (row == 2 and col == 2):
            fig.update_xaxes(title_text="Distance (AU)", row=row, col=col)

fig.update_layout(
    title="Segmented Spacetime Fields vs Distance from Sun",
    height=600,
    showlegend=False
)

fig.show()

## 8. φ-Spiral Analysis

Demonstrate the φ-based temporal scaling around Jupiter.

In [ ]:
# Find Jupiter's position
jupiter_idx = catalog[catalog['name'] == 'Jupiter'].index[0]
jupiter_pos = positions[jupiter_idx]

print(f"Jupiter position: {jupiter_pos} AU")

# Compute local time dilation at Jupiter
N_jupiter = field.compute_segment_density(jupiter_pos.reshape(1, -1))[0]
tau_jupiter = field.compute_time_dilation(np.array([N_jupiter]))[0]

print(f"Local segment density N = {N_jupiter:.4f}")
print(f"Local time dilation τ = {tau_jupiter:.4f}")
print(f"Time runs {1/tau_jupiter:.2f}x faster than baseline")

# Generate φ-spiral around Jupiter
from src.orbits import generate_phi_spiral

PHI = (1 + np.sqrt(5)) / 2
jupiter_radius_au = catalog.loc[jupiter_idx, 'radius_km'] / 149597870.7
spiral_radius = jupiter_radius_au * 50  # Scale up for visibility

spiral_points = generate_phi_spiral(
    center=jupiter_pos,
    radius=spiral_radius,
    tau_local=tau_jupiter,
    num_turns=PHI * 2,
    points_per_turn=50
)

# Visualize Jupiter and its φ-spiral
fig = go.Figure()

# Jupiter
fig.add_trace(go.Scatter3d(
    x=[jupiter_pos[0]], y=[jupiter_pos[1]], z=[jupiter_pos[2]],
    mode='markers',
    marker=dict(size=15, color='brown'),
    name='Jupiter'
))

# φ-spiral
fig.add_trace(go.Scatter3d(
    x=spiral_points[:, 0], y=spiral_points[:, 1], z=spiral_points[:, 2],
    mode='lines',
    line=dict(color='gold', width=4),
    name=f'φ-spiral (τ={tau_jupiter:.3f})'
))

fig.update_layout(
    title=f"φ-Spiral Clock around Jupiter (τ = {tau_jupiter:.3f})",
    scene=dict(
        aspectmode="data",
        xaxis_title="X (AU)",
        yaxis_title="Y (AU)", 
        zaxis_title="Z (AU)"
    ),
    height=500
)

fig.show()

print(f"\n✅ φ-spiral generated with {len(spiral_points)} points")

## 9. Export Results

Save the visualization and data for further analysis.

In [ ]:
# Save complete visualization
print("Saving visualization...")
visualizer.save_html("../demo_segmented_spacetime.html")

# Save field data
field_df = pd.DataFrame({
    'x_AU': vertices[:, 0],
    'y_AU': vertices[:, 1], 
    'z_AU': vertices[:, 2],
    'N': N,
    'tau': tau,
    'n': n
})

field_df.to_csv("../demo_field_data.csv", index=False)

print("\n📁 Files saved:")
print("   - demo_segmented_spacetime.html (interactive visualization)")
print("   - demo_field_data.csv (field values at mesh vertices)")

print("\n🎉 Demo complete! The segmented spacetime framework successfully:")
print("   ✅ Generated φ-based segment density fields")
print("   ✅ Computed time dilation τ(x) = φ^(-α·N(x))")
print("   ✅ Calculated refractive index n(x) = 1 + κ·N(x)")
print("   ✅ Created interactive 3D visualization")
print("   ✅ Demonstrated φ-spiral temporal clocks")